In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("vine").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://489325038812-bucket.s3.amazonaws.com/vine_header.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("vine_header.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [6]:
# retrieve all the rows where the total_votes count is equal to or greater than 20
total_votes_df = df.filter("total_votes>19")
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R2O4P39D0ALCQK|          2|           14|         43|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|     

In [10]:
# Create new DF where all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
vote_percentage_df = total_votes_df.withColumn("vote_percent", total_votes_df['helpful_votes']/total_votes_df['total_votes'])
helpful_df = vote_percentage_df.filter("vote_percent>=.5")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      vote_percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|0.8620689655172413|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|               1.0|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|0.9354838709677419|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|0.9629629629629629|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|0.9523809523809523|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|0.8974358974358975|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|               1.0|
|R1E1PQGHWGO3Q1|          1|  

In [11]:
# Retrieve all the rows where a review was written as part of the Vine program
vine_yes_df = helpful_df.filter("vine='Y'")
vine_yes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      vote_percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3KPC0NBUDASX3|          5|           25|         25|   Y|                N|               1.0|
|R119P2A95GGXX4|          5|           26|         28|   Y|                N|0.9285714285714286|
|R1HKIRME8AJ89Z|          5|           79|         82|   Y|                N|0.9634146341463414|
|R3FY3GMBGOBR22|          5|           12|         20|   Y|                N|               0.6|
|R15KH3FBSVYGBU|          5|           32|         37|   Y|                N|0.8648648648648649|
|R22XZEQLVLWW49|          5|           28|         33|   Y|                N|0.8484848484848485|
|R2OIQYVAL9D63R|          4|           59|         65|   Y|                N|0.9076923076923077|
|R1W3KCDHM6BMZT|          4|  

In [12]:
# Retrieve all the rows where a review was NOT written as part of the Vine program
vine_no_df = helpful_df.filter("vine='N'")
vine_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      vote_percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|0.8620689655172413|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|               1.0|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|0.9354838709677419|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|0.9629629629629629|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|0.9523809523809523|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|0.8974358974358975|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|               1.0|
|R1E1PQGHWGO3Q1|          1|  

##Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

### Paid

In [16]:
# Determine the total number of reviews
vine_yes_total = vine_yes_df.count()
print(vine_yes_total)

107


In [17]:
# Determine the total number of 5 star reviews
vine_yes_5 = vine_yes_df.filter('star_rating=5').count()
print(vine_yes_5)

56


In [19]:
# Determin Percentage of 5 star reviews
vine_yes_percent = vine_yes_5/vine_yes_total
print(vine_yes_percent)

0.5233644859813084


### Unpaid

In [20]:
# Determine the total number of reviews
vine_no_total = vine_no_df.count()
print(vine_no_total)

39869


In [21]:
# Determine the total number of 5 star reviews
vine_no_5 = vine_no_df.filter('star_rating=5').count()
print(vine_no_5)

21005


In [22]:
# Determin Percentage of 5 star reviews
vine_no_percent = vine_no_5/vine_no_total
print(vine_no_percent)

0.5268504351751988
